# Library for the Satellite images

> Indented block



In [1]:
!pip install earthpy

     |████████████████████████████████| 1.4 MB 5.1 MB/s 
     |████████████████████████████████| 19.3 MB 62.1 MB/s 
     |████████████████████████████████| 1.0 MB 71.3 MB/s 
     |████████████████████████████████| 6.3 MB 23.8 MB/s 
     |████████████████████████████████| 15.4 MB 32.8 MB/s 


In [2]:
pip install pyrsgis==0.3.9

## Data Prepration and pre-processing

In [4]:
# mnist_data = MNIST(root='./', download=True, transform=transforms.ToTensor())
import matplotlib.pyplot as plt
from pyrsgis import raster
from pyrsgis.ml import imageChipsFromArray, imageChipsFromFile
import torch.nn as nn
import torch

# read the TIF file(s) (both are of different sizes - for demonstration)
single_band_file = r'Playa_Image.tif' # this is a Landsat 5 TM image (7 bands stacked)
# multi_band_file = r'Playa_Image.tif' # this is a Landsat 5 TM image (7 bands stacked)

# create image chips
single_band_chips = imageChipsFromFile(single_band_file, x_size=28, y_size=28)
# multi_band_chips = imageChipsFromFile(multi_band_file, x_size=16, y_size=16)

print(single_band_chips.shape)
# print(multi_band_chips.shape)
# print(single_band_chips[1])

(189658, 28, 28, 7)


## Change datatype

In [5]:
import numpy as np
import cv2 
import tensorflow as tf


# changin the type of data images 
single_band_chips        = np.rollaxis(single_band_chips, 3, 1)
single_band_chips_float  = single_band_chips.astype(np.float32)

single_band_chips_tensor = torch.as_tensor(single_band_chips_float)

print(single_band_chips_tensor.dtype)

print(single_band_chips.shape)

torch.float32
(189658, 7, 28, 28)


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# from utils import ACTIVATION_DERIVATIVES
import math
import torch
# from flows import RadialFlow, PlanarFlow
from tqdm import tqdm
import torch.nn.functional as F
# from models import FCNEncoder, FCNDecoder, FlowModel
from torchvision.datasets import MNIST
from torchvision import transforms
from torch import optim
from torch.utils.data import DataLoader
from torch.nn import BCEWithLogitsLoss
from matplotlib import pyplot as plt
from scipy.special import logsumexp
import numpy as np
from sklearn.datasets import make_moons
from matplotlib import collections  as mc

## VAE Flow model

In [7]:
from typing import List
class FCNEncoder(nn.Module):
    def __init__(self, hidden_sizes: List[int], dim_input: int, activation=nn.ReLU()):
        super().__init__()
        
        
        hidden_sizes = [dim_input] + hidden_sizes
        
        self.net = []

        for i in range(len(hidden_sizes) - 1):
            self.net.append(nn.Linear(hidden_sizes[i], hidden_sizes[i+1]))
            self.net.append(nn.ReLU())
        
        self.net = nn.Sequential(*self.net)

        
    def forward(self, x):
        return self.net(x)

In [8]:
from torch.distributions.multivariate_normal import MultivariateNormal

class FlowModel(nn.Module):
    def __init__(self, flows: List[str], D: int, activation=torch.tanh):
        super().__init__()
        
        self.prior = MultivariateNormal(torch.zeros(D), torch.eye(D))
        self.net = []

        for i in range(len(flows)):
            layer_class = eval(flows[i])
            self.net.append(layer_class(D, activation))

        self.net = nn.Sequential(*self.net)

        self.D = D


    def forward(self, mu: torch.Tensor, log_sigma: torch.Tensor):
        """
        mu: tensor with shape (batch_size, D)
        sigma: tensor with shape (batch_size, D)
        """
        sigma = torch.exp(log_sigma)
        batch_size = mu.shape[0]
        samples = self.prior.sample(torch.Size([batch_size]))
        print(samples.size, sigma.shape)
        z = samples * sigma + mu

        z0 = z.clone().detach()
        log_prob_z0 = torch.sum(
            -0.5 * torch.log(torch.tensor(2 * math.pi)) - 
            log_sigma - 0.5 * ((z - mu) / sigma) ** 2, 
            axis=1)
        
        log_det = torch.zeros((batch_size,))
        
        for layer in self.net:
            z, ld = layer(z)
            log_det += ld

        log_prob_zk = torch.sum(
            -0.5 * (torch.log(torch.tensor(2 * math.pi)) + z ** 2), 
            axis=1)

        return z, log_prob_z0, log_prob_zk, log_det

In [12]:
ACTIVATION_DERIVATIVES = {
    F.elu: lambda x: torch.ones_like(x) * (x >= 0) + torch.exp(x) * (x < 0),
    torch.tanh: lambda x: 1 - torch.tanh(x) ** 2
}

class PlanarFlow(nn.Module):
    def __init__(self, D, activation=torch.tanh):
        super().__init__()
        self.D = D
        self.w = nn.Parameter(torch.empty(D))
        self.b = nn.Parameter(torch.empty(1))
        self.u = nn.Parameter(torch.empty(D))
        self.activation = activation
        self.activation_derivative = ACTIVATION_DERIVATIVES[activation]

        nn.init.normal_(self.w)
        nn.init.normal_(self.u)
        nn.init.normal_(self.b)

    def forward(self, z: torch.Tensor):
        lin = (z @ self.w + self.b).unsqueeze(1)  # shape: (B, 1)
        f = z + self.u * self.activation(lin)  # shape: (B, D)
        phi = self.activation_derivative(lin) * self.w  # shape: (B, D)
        log_det = torch.log(torch.abs(1 + phi @ self.u) + 1e-4) # shape: (B,)
        

        return f, log_det


class RadialFlow(nn.Module):
    def __init__(self, D, activation=torch.tanh):
        super().__init__()

        self.z0 = nn.Parameter(torch.empty(D))
        self.log_alpha = nn.Parameter(torch.empty(1))
        self.beta = nn.Parameter(torch.empty(1))
        self.activation = activation
        self.activation_derivative = ACTIVATION_DERIVATIVES[activation]
        self.D = D

        nn.init.normal_(self.z0) 
        nn.init.normal_(self.log_alpha)
        nn.init.normal_(self.beta)


    def forward(self, z: torch.Tensor):
        z_sub = z - self.z0
        alpha = torch.exp(self.log_alpha)
        r = torch.norm(z_sub)
        h = 1 / (alpha + r)
        f = z + self.beta * h * z_sub
        log_det = (self.D - 1) * torch.log(1 + self.beta * h) + \
            torch.log(1 + self.beta * h + self.beta - self.beta * r / (alpha + r) ** 2)

        return f, log_det

In [9]:
class FCNDecoder(nn.Module):
    def __init__(self, hidden_sizes: List[int], dim_input: int, activation=nn.ReLU()):
        super().__init__()
        
        hidden_sizes = [dim_input] + hidden_sizes
        self.net = []

        for i in range(len(hidden_sizes) - 1):
            self.net.append(nn.Linear(hidden_sizes[i], hidden_sizes[i+1]))
            self.net.append(nn.ReLU())
        
        self.net = nn.Sequential(*self.net)

    def forward(self, z: torch.Tensor):
        return self.net(z)

In [13]:
D = 40 # Depth (No. of channels)
encoder = FCNEncoder(hidden_sizes=[128, 64, 2*D], dim_input=7*28*28)
flow_model = FlowModel(flows=['PlanarFlow'] * 10, D=40)
decoder = FCNDecoder(hidden_sizes=[64, 128, 784], dim_input=40)
# optimizer Adam
optimizer = optim.Adam(list(encoder.parameters()) + list(flow_model.parameters()) + list(decoder.parameters()))

## Training of the model

In [15]:
loss_fn = BCEWithLogitsLoss()

for i, X_batch in enumerate(data_loader):
    optimizer.zero_grad()
    out = encoder(X_batch.view(-1, 7*784).float())
    mu, log_sigma = out[:, :40], out[:, 40:]
    z_k, log_prob_z0, log_prob_zk, log_det = flow_model(mu, log_sigma)
    x_hat = decoder(z_k)
    print(x_hat.shape, )
    
    loss = torch.mean(log_prob_z0) + loss_fn(x_hat, X_batch.view(-1, 7*784).float()) - torch.mean(log_prob_zk) - torch.mean(log_det)
    if i % 100 == 0:
        print(f'Iteration {i}, loss: {loss.item()}')
    
    loss.backward()
    optimizer.step()

<built-in method size of Tensor object at 0x7fc4414340b0> torch.Size([32, 40])


ValueError: ignored

In [ ]:
out = encoder(X_batch.view(-1, 784).float())
mu, log_sigma = out[:, :D], out[:, D:]
z_k, log_prob_z0, log_prob_zk, log_det = flow_model(mu, log_sigma)
x_hat = decoder(z_k)

loss = torch.mean(log_prob_z0) + loss_fn(x_hat, X_batch.view(-1, 784).float()) - torch.mean(log_prob_zk) - torch.mean(log_det)

# Trash

In [10]:
from torch.utils.data import Dataset
import numpy as np
import torch

class BinarizedMNIST(Dataset):
    def __init__(self, file):
        self.data = np.load(file)
        self.data = torch.tensor(self.data)

    def __len__(self,):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# mnist_data = MNIST(root='./', download=True, transform=transforms.ToTensor())
# data = BinarizedMNIST(single_band_chips)
data_loader = torch.utils.data.DataLoader(single_band_chips_tensor,
                                          batch_size=32,
                                          shuffle=True,
                                          num_workers=0)

In [3]:
from pyrsgis import raster
from pyrsgis.ml import imageChipsFromArray, imageChipsFromFile
import torch.nn as nn
import torch
# read the TIF file(s) (both are of different sizes - for demonstration)
single_band_file = r'Playa_Image.tif'
multi_band_file = r'Playa_Image.tif' # this is a Landsat 7 TM image (7 bands stacked)

# create image chips
single_band_chips = imageChipsFromFile(single_band_file, x_size=16, y_size=16)
multi_band_chips  = imageChipsFromFile(multi_band_file, x_size=16, y_size=16)

print(single_band_chips.shape)
print(multi_band_chips.shape)
# # read the files as array using pyrsgis raster.read module
# _, single_band_array = raster.read(single_band_file)
# _, multi_band_array = raster.read(multi_band_file)

# # create image chips
# single_band_chips = imageChipsFromArray(single_band_array, x_size=5, y_size=5)
# multi_band_chips  = imageChipsFromArray(multi_band_array, x_size=5, y_size=5)

# print(single_band_chips.shape)
# print(multi_band_chips.shape)

Warning! matplotlib_scalebar library not found. You may not be able to export map directly.
(189658, 16, 16, 7)
(189658, 16, 16, 7)
